## Airbnb Queries

This notebook shows the code used answer the three queries below.
1. Display list of stays in Portland, OR with details: name, neighbourhood, room type, how many guests it accommodates, property type and amenities, per night’s cost and is available for the next two days in descending order of rating.
2. Are there any neighbourhoods in any of the cities that don’t have any listings?
3. For each city, how many reviews are received for December of each year?

In [0]:
# Query 1 for list of stays in Portland with name, neighbourhood, room type, accomodates, property type, amenities, per night's cost and is available for next two days. Ordered by rating descending.

query1 = spark.sql("""SELECT name, neighbourhood_cleansed AS neighbourhood, room_type, accommodates, property_type, amenities, price, CASE
                                               WHEN availability_30 >= 2 THEN 'True'
                                               ELSE 'FALSE'
                                               END AS is_available_next_2_days
                      FROM global_temp.Portland_listings
                      ORDER BY review_scores_rating""")

query1.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query1.csv')

query1.show()

+--------------------+------------------+---------------+------------+--------------------+--------------------+-------+------------------------+
 name| neighbourhood| room_type|accommodates| property_type| amenities| price|is_available_next_2_days|
+--------------------+------------------+---------------+------------+--------------------+--------------------+-------+------------------------+
Light filled base...| Piedmont|Entire home/apt| 2| Entire guest suite|"""Coffee maker""...| $51.00| True|
Private Basement ...| Pleasant Valley| Private room| 3|Private room in r...|"""Clothing stora...| $70.00| True|
 Tabor Rest| Mt. Tabor|Entire home/apt| 4|Entire residentia...|"""Cooking basics...|$121.00| FALSE|
Sanctuary in the ...| Hollywood|Entire home/apt| 3|Entire condominiu...|"""Indoor firepla...| $70.00| True|
Charming Craftsma...| Humboldt|Entire home/apt| 4| Entire cottage|"""Free parking o...|$132.00| True|
Upscale “Pearl Pa...| Pearl|Entire home/apt| 3|Entire condominiu...|"""Single level h...|$150.00| True|
Bright & Clean Vi...| Buckman|Entire home/apt| 2| Entire rental unit|"""Portable fans"...| $78.00| True|
which features a ...| Portland Downtown| Private room| 1|Private room in r...|"""Kitchen""& ""C...| $70.00| True|
Executive Single ...| Reed|Entire home/apt| 3|Entire residentia...|"""Indoor firepla...|$100.00| FALSE|
Waterfront Pearl ...|Old Town/Chinatown|Entire home/apt| 6|Entire condominiu...|"""Dedicated work...|$145.00| True|
Belmont Laurelhur...| Sunnyside|Entire home/apt| 5|Entire residentia...|"""Shampoo""& ""E...|$145.00| True|
Portland House wi...| Forest Park|Entire home/apt| 5|Entire residentia...|"""Coffee maker""...|$250.00| FALSE|
30days+RemodeledC...| Grant Park|Entire home/apt| 16|Entire residentia...|"""Security camer...| $82.00| FALSE|
Resort Living in ...| Northwest Heights| Private room| 2|Private room in r...|"""TV""& ""Kitche...|$139.00| True|
PANO MT HOOD AND ...| South Portland| Private room| 2|Private room in r...|"""Shampoo""& ""E...|$199.00| True|
Hotel Rose, Water...| Portland Downtown| Private room| 2|Room in boutique ...|"""Elevator""& ""...|$120.00| True|
Light-filled char...| Buckman|Entire home/apt| 4|Entire residentia...|"""Outdoor dining...|$168.00| True|
Extended stay roo...| Buckman| Private room| 2|Private room in r...|"""Coffee maker""...| $46.00| FALSE|
Industrial Pearl ...| Pearl|Entire home/apt| 2|Entire condominiu...|"""Coffee maker""...|$150.00| True|
 Southwest Portland| Multnomah|Entire home/apt| 2| Entire rental unit|"""Essentials""& ...|$175.00| True|
+--------------------+------------------+---------------+------------+--------------------+--------------------+-------+------------------------+
only showing top 20 rows

In [0]:
# Query 2 for LA neighborhoods that don't have any listings

query2_LA = spark.sql("""SELECT neighbourhood AS LA_neighborhoods
                         FROM global_temp.LA_neighborhoods
                         WHERE neighbourhood NOT IN 
                                             (SELECT DISTINCT(neighbourhood_cleansed)
                                              FROM global_temp.LA_listings)""")

query2_LA.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query2_LA.csv')

query2_LA.show()

+----------------+
LA_neighborhoods|
+----------------+
 Hansen Dam|
 Littlerock|
 Walnut Park|
Whittier Narrows|
+----------------+

In [0]:
# Query 2 for Portland neighborhoods that don't have any listings

query2_Portland = spark.sql("""SELECT neighbourhood AS Portland_neighborhoods
                               FROM global_temp.Portland_neighborhoods
                               WHERE neighbourhood NOT IN 
                                                   (SELECT DISTINCT(neighbourhood_cleansed)
                                                    FROM global_temp.Portland_listings)""")

query2_Portland.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query2_Portland.csv')

query2_Portland.show()

+----------------------+
Portland_neighborhoods|
+----------------------+
 Northwest Industrial|
 Woodland Park|
+----------------------+

In [0]:
# Query 2 for Salem neighborhoods that don't have any listings

query2_Salem = spark.sql("""SELECT neighbourhood AS Salem_neighborhoods
                            FROM global_temp.Salem_neighborhoods
                            WHERE neighbourhood NOT IN 
                                                (SELECT DISTINCT(neighbourhood_cleansed)
                                                 FROM global_temp.Salem_listings)""")

query2_Salem.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query2_Salem.csv')

query2_Salem.show()

+-------------------+
Salem_neighborhoods|
+-------------------+
+-------------------+

In [0]:
# Query 2 for SD neighborhoods that don't have any listings

query2_SD = spark.sql("""SELECT neighbourhood AS SD_neighborhoods
                         FROM global_temp.SD_neighborhoods
                         WHERE neighbourhood NOT IN 
                                             (SELECT DISTINCT(neighbourhood_cleansed)
                                              FROM global_temp.SD_listings)""")

query2_SD.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query2_SD.csv')

query2_SD.show()

+----------------+
SD_neighborhoods|
+----------------+
 Bella Lago|
 Fenton St|
 Golf Course|
 Miramar|
 Thomy Locust Pl|
 Village Center|
+----------------+

In [0]:
# Query 3 for count of reviews received for listings in LA in December of each year

query3_LA = spark.sql("""SELECT COUNT(id) AS count_reviews, YEAR(date) AS year
                         FROM global_temp.LA_reviews
                         GROUP BY MONTH(date), YEAR(date)
                         HAVING MONTH(date) = 12
                         ORDER BY year""")

query3_LA.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query3_LA.csv')

query3_LA.show()

+-------------+----+
count_reviews|year|
+-------------+----+
 2|2009|
 14|2010|
 88|2011|
 237|2012|
 626|2013|
 1501|2014|
 3451|2015|
 5392|2016|
 9141|2017|
 14471|2018|
 19414|2019|
 10711|2020|
 27467|2021|
+-------------+----+

In [0]:
# Query 3 for count of reviews received for listings in Portland in December of each year

query3_Portland = spark.sql("""SELECT COUNT(id) AS count_reviews, YEAR(date) AS year
                               FROM global_temp.Portland_reviews
                               GROUP BY MONTH(date), YEAR(date)
                               HAVING MONTH(date) = 12
                               ORDER BY year""")

query3_Portland.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query3_Portland.csv')

query3_Portland.show()

+-------------+----+
count_reviews|year|
+-------------+----+
 7|2010|
 13|2011|
 58|2012|
 195|2013|
 416|2014|
 881|2015|
 1625|2016|
 2621|2017|
 4069|2018|
 5025|2019|
 3060|2020|
 6414|2021|
+-------------+----+

In [0]:
# Query 3 for count of reviews received for listings in Salem in December of each year

query3_Salem = spark.sql("""SELECT COUNT(id) AS count_reviews, YEAR(date) AS year
                            FROM global_temp.Salem_reviews
                            GROUP BY MONTH(date), YEAR(date)
                            HAVING MONTH(date) = 12
                            ORDER BY year""")

query3_Salem.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query3_Salem.csv')

query3_Salem.show()

+-------------+----+
count_reviews|year|
+-------------+----+
 7|2015|
 42|2016|
 60|2017|
 138|2018|
 188|2019|
 147|2020|
 109|2021|
+-------------+----+

In [0]:
# Query 3 for count of reviews received for listings in SD in December of each year

query3_SD = spark.sql("""SELECT COUNT(id) AS count_reviews, YEAR(date) AS year
                         FROM global_temp.SD_reviews
                         GROUP BY MONTH(date), YEAR(date)
                         HAVING MONTH(date) = 12
                         ORDER BY year""")

query3_SD.write.format('com.databricks.spark.csv').option('header',True).save('dbfs:/FileStore/query_results/query3_SD.csv')

query3_SD.show()

+-------------+----+
count_reviews|year|
+-------------+----+
 5|2010|
 13|2011|
 67|2012|
 199|2013|
 487|2014|
 1125|2015|
 2172|2016|
 3416|2017|
 5920|2018|
 7960|2019|
 6838|2020|
 5081|2021|
+-------------+----+